In [69]:
import json
import numpy as np
import igraph as ig
import plotly.graph_objects as go
from sklearn.datasets import fetch_california_housing


def create_uber_graph(file_geo: str, file_time: str) -> ig.Graph:
    """Returns an giant connected component of the uber movement data

    Parameters
    ----------
    file_geo : str
        path to the uber .json geo-json file
    file_time : str
        path to the uber uber drive times file .csv time file

    Returns
    -------
    g : ig.Graph
        giant connected component from the uber movement data
    """
    if not file_geo.endswith('.json'):
        raise ValueError('file_geo must be a .json file')
    if not file_time.endswith('.csv'):
        raise ValueError('file_time must be a .csv file')

    # load geography data as nodes 
    with open(file_geo) as f:
        census_tracts = json.loads(f.readline())

    display_names = dict()
    coordinates = dict()
    
    for area in census_tracts['features']:
        id = int(area['properties']['MOVEMENT_ID'])
        display_names[id] = area['properties']['DISPLAY_NAME''']
        a = area['geometry']['coordinates'][0]
        coordinates[id] = np.array(a if type(a[0][0]) == float else a[0]).mean(axis=0)[::-1]

    g = ig.Graph(directed=False)
    g.add_vertices(len(display_names))
    g.vs['display_name'] = list(display_names.values())  # index = id - 1
    g.vs['coordinates'] = list(coordinates.values())


    # Load drive time data as edges
    december_month = 12  # for monthly aggregate data of 4th quarter, we can filter data based off of only December

    edges = []
    weights = []

    with open(file_time) as f:
        f.readline()  # skip the first line

        for line in f:
            vals = line.strip().split(',')
            
            # read edge info
            src, dest, month, dist = int(vals[0]), int(vals[1]), int(vals[2]), float(vals[3])

            if month == december_month:
                edges.append((src - 1, dest - 1))
                weights.append(dist)

    g.add_edges(edges)
    g.es['weight'] = weights

    # keep only the giant connected component
    gcc = max(g.components(), key=len)
    g = g.subgraph(gcc)

    # remove duplicate edges
    return g.simplify(combine_edges=dict(weight='mean'))

Takes the graph G, finds the minimum spanning tree, clusters the MST.  
Each cluster is then searched for the node with the highest 'closeness centrality" to indicate the best location to station first responders.  
  
TODO:
- show that the center node is really the best location

# First Responders Station Locations

In [173]:
file_geo = "./data/los_angeles_censustracts.json"
file_time = "./data/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv"
g = create_uber_graph(file_geo, file_time)
print(f'Number of nodes: {len(g.vs)}', f'Number of edges: {len(g.es)}', sep='\n')

Number of nodes: 2649
Number of edges: 1003858


In [ ]:
# cluster graph into n clusters
n_clusters = 50
mst = g.spanning_tree(weights=g.es['weight'])
clusters = mst.community_fastgreedy(weights='weight').as_clustering(n=n_clusters)
g.vs['cluster'] = clusters.membership

# Find the nodes in each cluster that are closest to all other nodes in the cluster
# ie have the highest closeness centrality
cluster_membership = np.array(g.vs['cluster'])
cluster_mst_root_nodes = []
for i in range(n_clusters):
    cluster_nodes = np.where(cluster_membership == i)[0]
    subgraph = g.subgraph(cluster_nodes)
    min_node = None
    min_dist = float('inf')
    for node in subgraph.vs:
        dist_sum = 0
        for neighbor in subgraph.neighbors(node):
            dist_sum += subgraph.es[subgraph.get_eid(node, neighbor)]['weight']
            if dist_sum > min_dist:
                break
        if dist_sum < min_dist:
            min_dist = dist_sum
            min_node = node
    cluster_mst_root_nodes.append(min_node)

In [ ]:
vertices = g.vs
coordinates = np.array([v['coordinates'] for v in vertices])
colors = np.array([v['cluster'] for v in vertices])

# plot
fig = go.Figure(go.Scattermapbox(
    mode = "markers+text",
    lat = coordinates[:, 0],
    lon = coordinates[:, 1],
    marker = {'size': 7, 'color': colors},
    text = [f'{v.index}: {v["cluster"]}' for v in vertices],
    textposition = "bottom right",
    textfont = {'color': 'black'},
))

# plot cluster_mst-root nodes
colors = np.array([v['cluster'] for v in cluster_mst_root_nodes])
coordinates = np.array([v['coordinates'] for v in cluster_mst_root_nodes])
fig.add_trace(go.Scattermapbox(
    mode = "markers+text",
    lat = coordinates[:, 0],
    lon = coordinates[:, 1],
    marker = {'size': 20, 'color': colors},
    text = [f'{v.index}: {v["cluster"]}' for v in cluster_mst_root_nodes],
    textposition = "bottom right",
    textfont = {'color': 'black'},
))

fig.update_layout(
    mapbox = {
        'style': "carto-positron",
        'zoom': 9,
        'center': {'lon': -118.2437, 'lat': 34.0522},
    },
    margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
    height = 600,
)

fig.show()

# Best Place to Create a New UCLA Dorm
Based on a California Housing prices dataset and UCLA's location, we can find the best place to build a new dorm.

This dataset is from 1990 so the predictions are a bit outdated

In [132]:
# Manually obtained by taking max/min of all graph_lat_long values in a previous run
# min/max lat/long values of all coordinates in the Uber LA dataset
MIN_LAT = 33.6
MAX_LAT = 34.0
MIN_LON = -118.7
MAX_LON = -117.7
UCLA_IDX = 1523

file_geo = "./data/los_angeles_censustracts.json"
file_time = "./data/los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv"
g = create_uber_graph(file_geo, file_time)
graph_lat_longs = np.array([v['coordinates'] for v in g.vs])
print(f'Number of nodes: {len(g.vs)}', f'Number of edges: {len(g.es)}', sep='\n')

Number of nodes: 2649
Number of edges: 1003858


In [175]:
california_housing = fetch_california_housing(as_frame=True)

# filter out values in california housing that are within MIN_LAT, MAX_LAT, MIN_LON, MAX_LON
house_lat = california_housing.data['Latitude']
house_lon = california_housing.data['Longitude']
mask = (house_lat > MIN_LAT) & (house_lat < MAX_LAT) & (house_lon > MIN_LON) & (house_lon < MAX_LON)
house_lat = house_lat[mask].values
house_lon = house_lon[mask].values
house_med_value = california_housing.target[mask].values
housing_lat_longs = np.array([house_lat, house_lon]).T
print('N housing points in LA:', len(house_lat))

# Add prices form california housing to the graph
prices = []
for i, lat_lon in enumerate(graph_lat_longs):
    house_idx = np.argmin(np.linalg.norm(lat_lon - housing_lat_longs, axis=1))
    prices.append(house_med_value[house_idx])
prices = np.array(prices)

g.vs['price'] = prices

N housing points in LA: 3602


In [164]:
# Run DFS to find the nodes  within a certain radius in time of UCLA
# each DFS path length is capped at radius

def dfs(g: ig.Graph, root_node: int, radius: float):
    n_vertices = len(g.vs)
    visited = np.zeros(n_vertices, dtype=bool)
    nodes_in_radius = np.zeros(n_vertices, dtype=bool)
    visited[root_node] = True
    nodes_in_radius[root_node] = True
    adj = np.zeros((n_vertices, n_vertices))
    neigbhors = [[] for _ in range(n_vertices)]
    for u in range(n_vertices):
        for v in g.neighbors(u):
            neigbhors[u].append(v)
            val = g.es[g.get_eid(u, v)]['weight']
            adj[u, v] = val
            adj[v, u] = val

    stack = [(root_node, 0)] # node, path_len
    while stack:
        v, path_len = stack.pop()
        for neighbor in neigbhors[v]:
            new_path_len = path_len + adj[v, neighbor]
            if not visited[neighbor]:
                visited[neighbor] = True
                if new_path_len < radius:
                    nodes_in_radius[neighbor] = True
                    stack.append((neighbor, new_path_len))

    return nodes_in_radius

n_cheapest_nodes = 10
graph_inds = np.arange(len(g.vs))
nodes_in_radius_dict = {10: None, 30: None, 60: None}

for radius in nodes_in_radius_dict:
    nodes_in_radius = dfs(g, UCLA_IDX, 60*radius)
    inds = np.argsort(prices[nodes_in_radius])
    cheapest_nodes = graph_inds[nodes_in_radius][inds[:n_cheapest_nodes]]
    nodes_in_radius_dict[radius] = (nodes_in_radius, cheapest_nodes)

    print('Cheapest nodes:', cheapest_nodes)
    print(f'Number of nodes in a {radius} minute radius:', len(nodes_in_radius.nonzero()[0]))

Cheapest nodes: [ 112 1521 1522 1523 1530 1529 1504 1516 1533 1532]
Number of nodes in a 10 minute radius: 22
Cheapest nodes: [1293 1355 1354 1348 1372 1370 1333 1290 1334 1324]
Number of nodes in a 30 minute radius: 405
Cheapest nodes: [2605 2604 1470 2345 2349 1613 1484 1486 1442 1449]
Number of nodes in a 60 minute radius: 1305


The displayed plot shows the cheapest houses as purple and the most expensive ones as yellow. The big red dot is UCLA's location, and the small red dot is the best place to build a new dorm.

In [176]:
# plot graph_lat_long coordinates, with color corresponding to price and a colorbar
fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lat = graph_lat_longs[:, 0],
    lon = graph_lat_longs[:, 1],
    marker = {'size': 7, 'color': prices, 'colorscale': 'Viridis'},
    text = [f'{v.index}: {v["price"]}' for v in g.vs],
    textposition = "bottom right",
    textfont = {'color': 'black'},
))

# plot UCLA
fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lat = [graph_lat_longs[UCLA_IDX, 0]],
    lon = [graph_lat_longs[UCLA_IDX, 1]],
    marker = {'size': 20, 'color': 'red'},
    text = [f'{UCLA_IDX}: {g.vs[UCLA_IDX]["price"]}'],
    textposition = "bottom right",
    textfont = {'color': 'black'},
))

min_within_ucla = 30 # 10, 30, or 60
nodes_in_radius, cheapest_nodes = nodes_in_radius_dict[min_within_ucla]
lat_close_ucla = graph_lat_longs[cheapest_nodes, 0]
lon_close_ucla = graph_lat_longs[cheapest_nodes, 1]
prices_close_ucla = prices[cheapest_nodes]
fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lat = lat_close_ucla,
    lon = lon_close_ucla,
    marker = {'size': 7, 'color': 'red'},
    text = [f'{i}: {p}' for i, p in enumerate(prices_close_ucla)],
    textposition = "bottom right",
    textfont = {'color': 'black'},
))

fig.update_layout(
    mapbox = {
        'style': "carto-positron",
        'zoom': 9,
        'center': {'lon': -118.2437, 'lat': 34.0522},
    },
    margin = {'l': 0, 'r': 0, 'b': 0, 't': 0},
    height = 600,
)

fig.show()